# Module 06: Production Deployment - Follow Along

**Key Topics:** LangSmith monitoring, Platform deployment, Security, Scaling

Run production deployment examples.


In [ ]:
%pip install -q -U langgraph langsmith

import os
from langsmith import traceable
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

print('✅ Production tools ready!')

## Example 1: LangSmith Tracing


In [ ]:
# Enable LangSmith tracing
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'production-demo'
# os.environ['LANGSMITH_API_KEY'] = 'your-key-here'

class TracedState(TypedDict):
    input: str
    output: str

@traceable(name='critical_processing', tags=['production', 'monitored'])
def critical_node(state: TracedState):
    """This node is automatically traced to LangSmith."""
    result = f"Processed: {state['input']}"
    return {'output': result}

workflow = StateGraph(TracedState)
workflow.add_node('process', critical_node)
workflow.add_edge(START, 'process')
workflow.add_edge('process', END)

app = workflow.compile()
result = app.invoke({'input': 'test data', 'output': ''})

print(f"Result: {result['output']}")
print('✅ Trace visible at https://smith.langchain.com')
print('View: execution time, state changes, errors')

## Example 2: Deployment Configuration


In [ ]:
import json

# LangGraph Platform config (langgraph.json)
platform_config = {
    'dependencies': [
        'langgraph',
        'langchain-openai',
        'python-dotenv'
    ],
    'graphs': {
        'my_agent': './src/agent.py:graph'
    },
    'env': {
        'OPENAI_API_KEY': '$OPENAI_API_KEY'
    }
}

# Kubernetes Helm values
helm_values = {
    'replicaCount': 3,
    'image': {'repository': 'myorg/agent', 'tag': 'v1.0.0'},
    'resources': {
        'requests': {'memory': '2Gi', 'cpu': '1000m'},
        'limits': {'memory': '4Gi', 'cpu': '2000m'}
    },
    'autoscaling': {
        'enabled': True,
        'minReplicas': 3,
        'maxReplicas': 10,
        'targetCPUUtilizationPercentage': 70
    }
}

print('Platform config:', json.dumps(platform_config, indent=2))
print('\nHelm values:', json.dumps(helm_values, indent=2))
print('✅ Production deployment configs!')

## Example 3: Security - JWT Authentication


In [ ]:
# Production security pattern
security_code = '''
from fastapi import FastAPI, Depends, HTTPException, Security
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
import jwt

app = FastAPI()
security = HTTPBearer()
SECRET_KEY = os.getenv('JWT_SECRET_KEY')

def verify_token(credentials: HTTPAuthorizationCredentials = Security(security)):
    token = credentials.credentials
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=['HS256'])
        return payload
    except jwt.InvalidTokenError:
        raise HTTPException(status_code=401, detail='Invalid token')

@app.post('/invoke')
async def invoke_agent(request: dict, auth = Depends(verify_token)):
    user_id = auth.get('user_id')
    result = app.invoke(request)
    return {'user_id': user_id, 'result': result}
'''

print("Production security pattern:")
print(security_code)
print('\n✅ JWT authentication for API endpoints')

## Example 4: Monitoring & Alerts


In [ ]:
# Production monitoring pattern
monitoring_code = '''
import logging
from prometheus_client import Counter, Histogram

# Metrics
requests_total = Counter('agent_requests_total', 'Total requests')
request_duration = Histogram('agent_request_duration_seconds', 'Request duration')
errors_total = Counter('agent_errors_total', 'Total errors')

def monitored_node(state):
    requests_total.inc()
    
    with request_duration.time():
        try:
            result = process(state)
            return result
        except Exception as e:
            errors_total.inc()
            logging.error(f"Error: {e}")
            raise
'''

print("Monitoring with Prometheus:")
print(monitoring_code)
print('\n✅ Metrics: request count, duration, errors')

## Example 5: Production Checklist


In [ ]:
# Production readiness checklist
checklist = {
    'Monitoring': {
        'LangSmith tracing': True,
        'Prometheus metrics': True,
        'Error tracking': True
    },
    'Security': {
        'Authentication (JWT)': True,
        'Secrets encrypted': True,
        'Network policies': True
    },
    'Scalability': {
        'Auto-scaling enabled': True,
        'Load tested': True,
        'Persistent checkpointer': True
    },
    'Reliability': {
        'Error handling': True,
        'Retry logic': True,
        'Circuit breakers': True
    }
}

for category, checks in checklist.items():
    print(f'\n{category}:')
    for check, status in checks.items():
        icon = '✅' if status else '❌'
        print(f'  {icon} {check}')

print('\n✅ PRODUCTION READY!')

## Summary

You've run examples for:
- ✅ LangSmith tracing and monitoring
- ✅ Deployment configurations (Platform, K8s)
- ✅ Security (JWT authentication)
- ✅ Monitoring with Prometheus
- ✅ Production readiness checklist

**Next:** `module-06-practice.ipynb` for deployment exercises!
